In [1]:
from new_stations import *

In [ ]:
data, weather = load_data()

In [ ]:
data = feature_addition(data)

In [ ]:
sf = model_city(data,1)

In [ ]:
def month_sep(df, year, month):
    cdf = df[(df.year == year) & (df.month == month)]
    ndf = df[(df.year == year) & (df.month == (month-1))]
    return cdf, ndf

In [ ]:
cdf, ndf = month_sep(sf, 2017, 12)

In [ ]:
ps = new_stn_coords(cdf, ndf)

In [ ]:
ps

In [ ]:
knn_proposed_stn(cdf, ndf, ps)